In [153]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re
import bs4
import requests
from collections import defaultdict

In [157]:
sites_to_scrape = ["https://www.instagram.com/explore/locations/6889842/paris-france/",
                   "https://www.instagram.com/explore/locations/213385402/london-united-kingdom/",
                   "https://www.instagram.com/explore/locations/118001404/copenhagen-denmark/",
                   "https://www.instagram.com/explore/locations/212988663/new-york-new-york/",
                   "https://www.instagram.com/explore/locations/116231/amsterdam-netherlands/",
                   "https://www.instagram.com/explore/locations/274029466/singapore/"]

scroll = 25
allLinks = set()
userStore = defaultdict(list)

In [158]:
path_to_chromedriver = 'C:\\Users\\njacobsen\\Documents\\Chrome Driver\\chromedriver.exe'
driver = webdriver.Chrome(executable_path = path_to_chromedriver)
driver.get(sites_to_scrape[0])

for i in range(scroll):
    try:
        driver.execute_script("window.scrollBy(0,500)")
    except:
        pass
    time.sleep(2)
    scrapeLinks = [x.get_attribute('href') for x in driver.find_elements_by_css_selector('a')]
    cleanLinks = [x for x in scrapeLinks if x != None and "/p/" in x]
    allLinks.update(cleanLinks)

In [172]:
for i in allLinks:
    driver.get(i)
    try:
        likes = re.sub("\D","",driver.find_elements_by_class_name("_nzn1h")[0].text)
    except:
        likes = 0

    soup = BeautifulSoup(driver.page_source, "lxml")
    userName = [x.get_attribute('href') for x in driver.find_elements_by_css_selector('a')][0][26:-1]

    image_list = [] # create empty list to save img urls
    for link in soup.find_all('img'): # find img tags in HTML
        m = re.search('src="(.+?)"', str(link))
        image_list.append(m.group(1)) # append url path to list

    try:
        img = image_list[1]
        userStore[userName].append([img, "paris", likes])
    except:
        pass